*Click [me](https://git.ligo.org/publications/O3/o3-cbc-burst-imbh/-/wikis/IMBH-rate-estimation#rate-estimation) to go to review page:*

### Calculating the Rates

The posterior on $\Lambda = R \langle VT \rangle$ for $n$ candidates can be expressed as:

$p(\Lambda|d) \propto p(R|\langle VT \rangle) p(\langle VT \rangle)e^{-\Lambda} \Lambda^n$

which when marginalised over $\langle VT \rangle$ gives $p(R|d)$. The upper limit $R_c$ on the rate with confidence $c$ is then the solution to:

$\int^{R_c}_0 dR p(R|d) = c$

For uniform prior on $\Lambda$, the rate at $90\%$ confidence is:
* $R^{n=0}_{90\%} = \frac{2.303}{\langle VT \rangle}$
* $R^{n=1}_{90\%} = \frac{3.88972}{\langle VT \rangle}$


* Please note here I am using the case of OR injections $\rightarrow$ meaning we are using union of injection time from both the pipeline runs

In [1]:
%pylab inline
%config InlineBackend.figure_format = 'retina'

Populating the interactive namespace from numpy and matplotlib


### Reading Modules

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy
from scipy.integrate import trapz, cumtrapz
import json
import numpy
import seaborn as sns
from scipy.integrate import cumtrapz
import pylab
pylab.style.use('seaborn-colorblind')

### Creating the Dataframe containing Injection bin wise Results

The lists in the following cells are manually put in by first downloading the `README*log` from [this git repository](https://git.ligo.org/imbhb/searches/-/tree/master/injections/O3) 

and then doing ` tail README_bins*log | grep -e "Surveyed spacetime volume:" -e "Total number of simulations before SNR threshold:" -e "Bin number:" -e "Max redshift:" -e "Mass bin:"`.

The outputs will be somewhat like this:
```
Mass bin: U_120M_7d0q_nospin_modes_up-to_l4
Bin number: 4
Max redshift: 0.9
Surveyed spacetime volume: 79.6862 Gpc^3 yr
Total number of simulations before SNR threshold: 297997
```
So:

* M_tot = 120
* q = 7
* chieff = 0
* chip = 0
* MassBin = 4
* z = 0.9
* N_tot = 297997
* V_tot = 79.6862

in the notebook. Further details regarding the waveforms can be found [here](https://wiki.ligo.org/Main/O3IMBH_XMLGenerationReview)

In [3]:
MassBin = numpy.arange(0,46)

M_tot = [120, 120, 120, 120, 120, 120, 150,
         200, 200, 200, 200, 220, 250, 300, 350,
         400, 400, 400, 400, 400, 440, 500, 600, 600, 800,
         120, 200, 400, 600, 800,
         120, 200, 400, 600, 800,
         200, 200, 200, 200, 
         400, 400, 400, 400, 600, 600, 800]

q = [1, 2, 4., 5, 7, 10, 2,
     1, 2, 4, 7, 10, 4, 2, 6,
     1, 2, 3, 4, 7, 10, 3/2, 1, 2, 1,
     1, 1, 1, 1, 1,
     1, 1, 1, 1, 1,
     1, 2, 4, 7,
     1, 2, 4, 7, 1, 2, 1]

chieff, chip = [], []
chieff += 25 * [0]
chieff += 5 * [0.8]
chieff += 5 * [-0.8]
chieff += [0.51, 0.14, 0.26, 0.32, 0.51, 0.14, 0.26, 0.32, 0.51, 0.14, 0.51]
BinKind = []
BinKind+= 46*['']
chip += 35 * [0]
chip += 11 * [0.42]


N_tot = [235438, 242893, 269575, 277772, 297997, 321276, 209975, 185246, 184217,
         193364, 201356, 192595, 171494, 157489, 141812, 170032, 144070, 124076,
         115944, 118781, 100269, 152056, 158088, 131182, 153438, 166859, 154551,
         139675, 138797, 148502, 253133, 205135, 187041, 180722, 175714, 155127,
         186574, 190711, 206275, 141121, 128394, 110405, 106686, 115607, 106521, 
         131695]
VT_tot = [340.078, 282.083, 162.518, 124.720, 79.686, 47.8479, 255.64, 255.64, 209.793,
          124.72, 63.1258, 34.1968, 97.2134, 124.72, 34.1968, 97.2134, 71.2662, 47.8479, 
          34.1968, 17.5403, 6.35876, 47.8479, 28.0926, 22.5271, 9.43448, 426.887, 399.686,
          200.413, 88.3472, 40.7967, 246.634, 246.634, 55.3055, 13.167, 3.94099, 307.595,
          264.552, 200.413, 124.72, 134.097, 106.251, 79.6861, 47.8478, 47.8478, 34.1968,
          17.5403]

In [4]:
df = pd.DataFrame()
df['MassBin'] = MassBin
df['Bin Kind'] = BinKind
df['M_tot'] = M_tot
df['q'] = q
df['chieff'] = chieff
df['chip'] = chip
df['N_tot'] = N_tot
df['VT_tot'] = VT_tot

In [5]:
sim_id=['SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905', 
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454',
        'SXS:BBH:0056, RIT:BBH:0120:n140, GT:0906',
        'SXS:BBH:0298 RIT:BBH:Q10:n173, GT:0568',
        'SXS:BBH:0154, RIT:BBH:0068:n100',
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905',
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454',
        'SXS:BBH:0298 RIT:BBH:Q10:n173, GT:0568',
        'SXS:BBH:0154, RIT:BBH:0068:n100',
        'SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454',
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0181, RIT:BBH:0121:n140, GT:0604',
        'SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905',
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0030, RIT:BBH:0102:n140, GT:0453',
        'SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454',
        'SXS:BBH:0298 RIT:BBH:Q10:n173, GT:0568',
        'RIT:BBH:Q10:n173, GT:0568',
        'RIT:BBH:0115:n140, GT:0477',
        'SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905',
        'SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446',
        'SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905',
        'SXS:BBH:0230, RIT:BBH:0063:n100',
        'SXS:BBH:0230, RIT:BBH:0063:n100',
        'SXS:BBH:0230, RIT:BBH:0063:n100',
        'SXS:BBH:0230, RIT:BBH:0063:n100',
        'SXS:BBH:0230, RIT:BBH:0063:n100',
        'SXS:BBH:0154,RIT:BBH:0068:n100',
        'SXS:BBH:0154,RIT:BBH:0068:n100',
        'SXS:BBH:0154,RIT:BBH:0068:n100',
        'SXS:BBH:0154,RIT:BBH:0068:n100',
        'SXS:BBH:0154,RIT:BBH:0068:n100',
        'GT:0803',
        'GT:0872',
        'GT:0875',
        'GT:0888',
        'GT:0803',
        'GT:0872',
        'GT:0875',
        'GT:0888',
        'GT:0803',
        'GT:0872',
        'GT:0803',
        ]

In [6]:
z = [2.35, 2.00, 1.35, 1.15, 0.90, 0.70, 1.85, 1.85,
     1.60, 1.15, 0.80, 0.60, 1.00 , 1.15, 0.60, 1.0,
     0.85, 0.70, 0.60, 0.45, 0.30, 0.70, 0.55, 0.50,
     0.35, 2.95, 2.75, 1.55, 0.95, 0.65, 1.80, 1.45,
     0.75, 0.40, 0.25, 2.15, 1.90, 1.55, 1.15, 1.20, 
     1.05, 0.90, 0.70, 0.70, 0.60, 0.45
     ]

In [7]:
dir = '/home/koustav.chandra/O3/rate_calculation/nr_rate_inj/common_inj/found_combined'
fap_threshold = 0.2531 # combined FAP of S200214br
fap_pipeline_threshold = 0.091982 # cWB FAP of S200214br
df['N_rec'] = numpy.nan
df['PyCBC_N_rec'] = numpy.nan
df['GstLAL_N_rec'] = numpy.nan
df['cWB_N_rec'] = numpy.nan
for i in range(len(df['MassBin'])):
    df_combined = pd.read_csv(dir+ '/combined_{}.csv'.format(df['MassBin'][i]),
                              delimiter=',',
                              usecols=['PyCBC FAP', 'cWB FAP', 'GstLAL FAP', 'Combined FAP'])
    df['N_rec'][i] = (df_combined[df_combined['Combined FAP'] <= fap_threshold]).shape[0]
    df['cWB_N_rec'][i] = (df_combined[df_combined['cWB FAP'] <= fap_pipeline_threshold]).shape[0]
    df['PyCBC_N_rec'][i] = (df_combined[df_combined['PyCBC FAP'] <= fap_pipeline_threshold]).shape[0]
    df['GstLAL_N_rec'][i] = (df_combined[df_combined['GstLAL FAP'] <= fap_pipeline_threshold]).shape[0]

In [8]:
df['SIM_ID'] = sim_id
df['z'] = z
#df['N_rec'] = N_rec
df['VT_sen'] = df['N_rec']/df['N_tot']*df['VT_tot']
df['R_n_0'] = 2.303/df['VT_sen']
df['R_n_1'] = 3.88972/df['VT_sen']

### Table containing all the details of the Injections Bin-Wise

In [9]:
df.to_csv('combined_pipelines.csv')

In [10]:
df2 = df[['MassBin','M_tot','q','chieff','chip','N_tot','VT_tot','N_rec','SIM_ID','z','VT_sen','R_n_0','R_n_1']]

In [11]:
df2

,MassBin,M_tot,q,chieff,chip,N_tot,VT_tot,N_rec,SIM_ID,z,VT_sen,R_n_0,R_n_1
0,0,120,1.0,0.00,0.00,235438,340.07800,12066.0,"SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905",2.35,17.428712,0.132138,0.223179
1,1,120,2.0,0.00,0.00,242893,282.08300,10820.0,"SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446",2.00,12.565772,0.183276,0.309549
2,2,120,4.0,0.00,0.00,269575,162.51800,8510.0,"SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454",1.35,5.130402,0.448893,0.758171
3,3,120,5.0,0.00,0.00,277772,124.72000,7763.0,"SXS:BBH:0056, RIT:BBH:0120:n140, GT:0906",1.15,3.485597,0.660719,1.115941
4,4,120,7.0,0.00,0.00,297997,79.68600,7006.0,"SXS:BBH:0298 RIT:BBH:Q10:n173, GT:0568",0.90,1.873442,1.229288,2.076242
5,5,120,10.0,0.00,0.00,321276,47.84790,6212.0,"SXS:BBH:0154, RIT:BBH:0068:n100",0.70,0.925158,2.489304,4.204383
6,6,150,2.0,0.00,0.00,209975,255.64000,10645.0,"SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446",1.85,12.960056,0.177700,0.300131
7,7,200,1.0,0.00,0.00,185246,255.64000,11778.0,"SXS:BBH:0180, RIT:BBH:0198:n140, GT:0905",1.85,16.253673,0.141691,0.239313
8,8,200,2.0,0.00,0.00,184217,209.79300,10400.0,"SXS:BBH:0169, RIT:BBH:0117:n140, GT:0446",1.60,11.843897,0.194446,0.328416
9,9,200,4.0,0.00,0.00,193364,124.72000,7510.0,"SXS:BBH:0182, RIT:BBH:0119:n140, GT:0454",1.15,4.843959,0.475438,0.803004
